In [13]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
import xgboost as xgb
import time
import pickle
from PIL import Image, ImageStat
from skimage import io
import cv2

import scipy
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
from multiprocessing import Pool

from PIL import Image

In [14]:
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

# Load data
train_path = './data/train-jpg/'
test_path = './data/test-jpg/'
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/sample_submission.csv')

In [15]:
def extract_features_thread(path):
	if '.jpg' not in path:
		path += '.jpg'
	st = []
	# pillow jpg
	img = Image.open(path)
	im_stats_ = ImageStat.Stat(img)
	st += im_stats_.sum
	st += im_stats_.mean
	st += im_stats_.rms
	st += im_stats_.var
	st += im_stats_.stddev
	img = np.array(img)[:, :, :3]
	st += [scipy.stats.kurtosis(img[:, :, 0].ravel())]
	st += [scipy.stats.kurtosis(img[:, :, 1].ravel())]
	st += [scipy.stats.kurtosis(img[:, :, 2].ravel())]
	st += [scipy.stats.skew(img[:, :, 0].ravel())]
	st += [scipy.stats.skew(img[:, :, 1].ravel())]
	st += [scipy.stats.skew(img[:, :, 2].ravel())]
	# cv2 jpg
	img = cv2.imread(path)
	bw = cv2.imread(path, 0)
	st += list(cv2.calcHist([bw], [0], None, [256], [0, 256]).flatten())  # bw
	st += list(cv2.calcHist([img], [0], None, [256], [0, 256]).flatten())  # r
	st += list(cv2.calcHist([img], [1], None, [256], [0, 256]).flatten())  # g
	st += list(cv2.calcHist([img], [2], None, [256], [0, 256]).flatten())  # b
	try:
		# skimage tif
		imgr = io.imread(path.replace('jpg', 'tif'))
		tf = imgr[:, :, 3]
		st += list(cv2.calcHist([tf], [0], None, [256], [0, 65536]).flatten())  # near ifrared
		ndvi = ((imgr[:, :, 3] - imgr[:, :, 0]) / (
		imgr[:, :, 3] + imgr[:, :, 0]))  # water ~ -1.0, barren area ~ 0.0, shrub/grass ~ 0.2-0.4, forest ~ 1.0
		st += list(np.histogram(ndvi, bins=20, range=(-1, 1))[0])
		ndvi = ((imgr[:, :, 3] - imgr[:, :, 1]) / (imgr[:, :, 3] + imgr[:, :, 1]))
		st += list(np.histogram(ndvi, bins=20, range=(-1, 1))[0])
		ndvi = ((imgr[:, :, 3] - imgr[:, :, 2]) / (imgr[:, :, 3] + imgr[:, :, 2]))
		st += list(np.histogram(ndvi, bins=20, range=(-1, 1))[0])
	except:
		st += [-1 for i in range(256)]
		st += [-2 for i in range(60)]
		print('err', path.replace('jpg', 'tif'))
	m, s = cv2.meanStdDev(img)  # mean and standard deviation
	st += list(m)
	st += list(s)
	st += [cv2.Laplacian(bw, cv2.CV_64F).var()]
	st += [cv2.Laplacian(img, cv2.CV_64F).var()]
	st += [cv2.Sobel(bw, cv2.CV_64F, 1, 0, ksize=5).var()]
	st += [cv2.Sobel(bw, cv2.CV_64F, 0, 1, ksize=5).var()]
	st += [cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5).var()]
	st += [cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5).var()]
	st += [(bw < 30).sum()]
	st += [(bw > 225).sum()]

	return st

def extract_features_multi(files, data_path):
	paths = [data_path + image_name for image_name in files]
	pool = Pool(12)
	X = pool.map(extract_features_thread, paths)
	print('Forming array')
	X = np.array(X)
	return X

In [16]:
def extract_features(df, data_path):
    im_features = df.copy()

    r_mean = []
    g_mean = []
    b_mean = []

    r_std = []
    g_std = []
    b_std = []

    r_max = []
    g_max = []
    b_max = []

    r_min = []
    g_min = []
    b_min = []

    r_kurtosis = []
    g_kurtosis = []
    b_kurtosis = []
    
    r_skewness = []
    g_skewness = []
    b_skewness = []
    
    r_hist = []
    g_hist = []
    b_hist = []

    for image_name in tqdm(im_features.image_name.values, miniters=100): 
        im = Image.open(data_path + image_name + '.jpg')
        img = cv2.imread(data_path + image_name + '.jpg')
        im = np.array(im)[:,:,:3]

        r_mean.append(np.mean(im[:,:,0].ravel()))
        g_mean.append(np.mean(im[:,:,1].ravel()))
        b_mean.append(np.mean(im[:,:,2].ravel()))

        r_std.append(np.std(im[:,:,0].ravel()))
        g_std.append(np.std(im[:,:,1].ravel()))
        b_std.append(np.std(im[:,:,2].ravel()))

        r_max.append(np.max(im[:,:,0].ravel()))
        g_max.append(np.max(im[:,:,1].ravel()))
        b_max.append(np.max(im[:,:,2].ravel()))

        r_min.append(np.min(im[:,:,0].ravel()))
        g_min.append(np.min(im[:,:,1].ravel()))
        b_min.append(np.min(im[:,:,2].ravel()))

        r_kurtosis.append(scipy.stats.kurtosis(im[:,:,0].ravel()))
        g_kurtosis.append(scipy.stats.kurtosis(im[:,:,1].ravel()))
        b_kurtosis.append(scipy.stats.kurtosis(im[:,:,2].ravel()))
        
        r_skewness.append(scipy.stats.skew(im[:,:,0].ravel()))
        g_skewness.append(scipy.stats.skew(im[:,:,1].ravel()))
        b_skewness.append(scipy.stats.skew(im[:,:,2].ravel()))
        
        r_hist.append(list(cv2.calcHist([img], [0], None, [256], [0, 256]).flatten()))
        g_hist.append(list(cv2.calcHist([img], [1], None, [256], [0, 256]).flatten()))
        b_hist.append(list(cv2.calcHist([img], [2], None, [256], [0, 256]).flatten()))
        
        
    im_features['r_mean'] = r_mean
    im_features['g_mean'] = g_mean
    im_features['b_mean'] = b_mean

    im_features['r_std'] = r_std
    im_features['g_std'] = g_std
    im_features['b_std'] = b_std

    im_features['r_max'] = r_max
    im_features['g_max'] = g_max
    im_features['b_max'] = b_max

    im_features['r_min'] = r_min
    im_features['g_min'] = g_min
    im_features['b_min'] = b_min

    im_features['r_kurtosis'] = r_kurtosis
    im_features['g_kurtosis'] = g_kurtosis
    im_features['b_kurtosis'] = b_kurtosis
    
    im_features['r_skewness'] = r_skewness
    im_features['g_skewness'] = g_skewness
    im_features['b_skewness'] = b_skewness
    
    im_features['r_hist'] = np.array(r_hist)
    im_features['g_hist'] = np.array(r_hist)
    im_features['b_hist'] = np.array(b_hist)
    
    return im_features


In [17]:
# Extract features
#print('Extracting train features')
#train_features = extract_features(train, train_path)
#print('Extracting test features')
#test_features = extract_features(test, test_path)

In [ ]:
print('Extracting train features')
t0 = time.time()
train = train.image_name.values
X = extract_features_multi(train, train_path)
print(time.time() - t0)
# Prepare data
#X = np.array(train_features.drop(['image_name', 'tags'], axis=1))
y_train = []

flatten = lambda l: [item for sublist in l for item in sublist]
labels = np.array(list(set(flatten([l.split(' ') for l in train_features['tags'].values]))))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}


Extracting train features
('err', './data/train-tif/train_1688.tif')
('err', './data/train-tif/train_844.tif')
('err', './data/train-tif/train_0.tif')
('err', './data/train-tif/train_2532.tif')
('err', './data/train-tif/train_3376.tif')
('err', './data/train-tif/train_5064.tif')
('err', './data/train-tif/train_5908.tif')
('err', './data/train-tif/train_4220.tif')
('err', './data/train-tif/train_6752.tif')
('err', './data/train-tif/train_7596.tif')
('err', './data/train-tif/train_8440.tif')
('err', './data/train-tif/train_9284.tif')
('err', './data/train-tif/train_1.tif')
('err', './data/train-tif/train_3377.tif')
('err', './data/train-tif/train_4221.tif')
('err', './data/train-tif/train_845.tif')
('err', './data/train-tif/train_9285.tif')
('err', './data/train-tif/train_6753.tif')
('err', './data/train-tif/train_5065.tif')
('err', './data/train-tif/train_7597.tif')
('err', './data/train-tif/train_5909.tif')
('err', './data/train-tif/train_2533.tif')
('err', './data/train-tif/train_1689

('err', './data/train-tif/train_6769.tif')
('err', './data/train-tif/train_3391.tif')
('err', './data/train-tif/train_1703.tif')
('err', './data/train-tif/train_859.tif')
('err', './data/train-tif/train_9298.tif')
('err', './data/train-tif/train_4236.tif')
('err', './data/train-tif/train_5081.tif')
('err', './data/train-tif/train_5926.tif')
('err', './data/train-tif/train_8456.tif')
('err', './data/train-tif/train_1704.tif')
('err', './data/train-tif/train_18.tif')
('err', './data/train-tif/train_6770.tif')
('err', './data/train-tif/train_860.tif')
('err', './data/train-tif/train_9299.tif')
('err', './data/train-tif/train_2549.tif')
('err', './data/train-tif/train_7610.tif')
('err', './data/train-tif/train_4237.tif')
('err', './data/train-tif/train_19.tif')
('err', './data/train-tif/train_2550.tif')
('err', './data/train-tif/train_8457.tif')
('err', './data/train-tif/train_7611.tif')
('err', './data/train-tif/train_5927.tif')
('err', './data/train-tif/train_9300.tif')
('err', './data/t

('err', './data/train-tif/train_5098.tif')
('err', './data/train-tif/train_4252.tif')
('err', './data/train-tif/train_9315.tif')
('err', './data/train-tif/train_3406.tif')
('err', './data/train-tif/train_5099.tif')
('err', './data/train-tif/train_7624.tif')
('err', './data/train-tif/train_34.tif')
('err', './data/train-tif/train_6786.tif')
('err', './data/train-tif/train_5942.tif')
('err', './data/train-tif/train_1720.tif')
('err', './data/train-tif/train_2564.tif')
('err', './data/train-tif/train_5100.tif')
('err', './data/train-tif/train_4253.tif')
('err', './data/train-tif/train_874.tif')
('err', './data/train-tif/train_9316.tif')
('err', './data/train-tif/train_8473.tif')
('err', './data/train-tif/train_7625.tif')
('err', './data/train-tif/train_3407.tif')
('err', './data/train-tif/train_35.tif')
('err', './data/train-tif/train_6787.tif')
('err', './data/train-tif/train_5101.tif')
('err', './data/train-tif/train_2565.tif')
('err', './data/train-tif/train_1721.tif')
('err', './data/

('err', './data/train-tif/train_1735.tif')
('err', './data/train-tif/train_5956.tif')
('err', './data/train-tif/train_2581.tif')
('err', './data/train-tif/train_5115.tif')
('err', './data/train-tif/train_3423.tif')
('err', './data/train-tif/train_7642.tif')
('err', './data/train-tif/train_8487.tif')
('err', './data/train-tif/train_4269.tif')
('err', './data/train-tif/train_9332.tif')
('err', './data/train-tif/train_2582.tif')
('err', './data/train-tif/train_5116.tif')
('err', './data/train-tif/train_889.tif')
('err', './data/train-tif/train_6801.tif')
('err', './data/train-tif/train_1736.tif')
('err', './data/train-tif/train_5957.tif')
('err', './data/train-tif/train_8488.tif')
('err', './data/train-tif/train_7643.tif')
('err', './data/train-tif/train_2583.tif')
('err', './data/train-tif/train_890.tif')
('err', './data/train-tif/train_5117.tif')
('err', './data/train-tif/train_50.tif')
('err', './data/train-tif/train_4270.tif')
('err', './data/train-tif/train_9333.tif')
('err', './data

('err', './data/train-tif/train_1751.tif')
('err', './data/train-tif/train_7659.tif')
('err', './data/train-tif/train_2597.tif')
('err', './data/train-tif/train_6817.tif')
('err', './data/train-tif/train_63.tif')
('err', './data/train-tif/train_9346.tif')
('err', './data/train-tif/train_5132.tif')
('err', './data/train-tif/train_905.tif')
('err', './data/train-tif/train_5971.tif')
('err', './data/train-tif/train_4286.tif')
('err', './data/train-tif/train_8506.tif')
('err', './data/train-tif/train_3439.tif')
('err', './data/train-tif/train_9347.tif')
('err', './data/train-tif/train_2598.tif')
('err', './data/train-tif/train_5133.tif')
('err', './data/train-tif/train_906.tif')
('err', './data/train-tif/train_6818.tif')
('err', './data/train-tif/train_4287.tif')
('err', './data/train-tif/train_7660.tif')
('err', './data/train-tif/train_5972.tif')
('err', './data/train-tif/train_64.tif')
('err', './data/train-tif/train_3440.tif')
('err', './data/train-tif/train_9348.tif')
('err', './data/t

...

('err', './data/train-tif/train_14411.tif')
('err', './data/train-tif/train_11341.tif')
('err', './data/train-tif/train_13627.tif')
('err', './data/train-tif/train_12090.tif')
('err', './data/train-tif/train_10618.tif')
('err', './data/train-tif/train_12832.tif')
('err', './data/train-tif/train_14412.tif')
('err', './data/train-tif/train_15202.tif')
('err', './data/train-tif/train_13628.tif')
('err', './data/train-tif/train_10619.tif')
('err', './data/train-tif/train_11342.tif')
('err', './data/train-tif/train_12091.tif')
('err', './data/train-tif/train_14413.tif')
('err', './data/train-tif/train_12833.tif')
('err', './data/train-tif/train_13629.tif')
('err', './data/train-tif/train_10620.tif')
('err', './data/train-tif/train_11343.tif')
('err', './data/train-tif/train_15203.tif')
('err', './data/train-tif/train_12092.tif')
('err', './data/train-tif/train_13630.tif')
('err', './data/train-tif/train_10621.tif')
('err', './data/train-tif/train_15204.tif')
('err', './data/train-tif/train_

('err', './data/train-tif/train_10644.tif')
('err', './data/train-tif/train_14434.tif')
('err', './data/train-tif/train_13655.tif')
('err', './data/train-tif/train_12854.tif')
('err', './data/train-tif/train_15227.tif')
('err', './data/train-tif/train_11365.tif')
('err', './data/train-tif/train_16880.tif')
('err', './data/train-tif/train_12114.tif')
('err', './data/train-tif/train_16051.tif')
('err', './data/train-tif/train_10645.tif')
('err', './data/train-tif/train_13656.tif')
('err', './data/train-tif/train_14435.tif')
('err', './data/train-tif/train_12855.tif')
('err', './data/train-tif/train_15228.tif')
('err', './data/train-tif/train_16881.tif')
('err', './data/train-tif/train_11366.tif')
('err', './data/train-tif/train_16052.tif')
('err', './data/train-tif/train_12115.tif')
('err', './data/train-tif/train_10646.tif')
('err', './data/train-tif/train_16053.tif')
('err', './data/train-tif/train_12856.tif')
('err', './data/train-tif/train_12116.tif')
('err', './data/train-tif/train_

('err', './data/train-tif/train_15245.tif')
('err', './data/train-tif/train_12136.tif')
('err', './data/train-tif/train_14452.tif')
('err', './data/train-tif/train_13673.tif')
('err', './data/train-tif/train_17737.tif')
('err', './data/train-tif/train_16073.tif')
('err', './data/train-tif/train_11385.tif')
('err', './data/train-tif/train_12870.tif')
('err', './data/train-tif/train_10663.tif')
('err', './data/train-tif/train_16901.tif')
('err', './data/train-tif/train_14453.tif')
('err', './data/train-tif/train_17738.tif')
('err', './data/train-tif/train_12137.tif')
('err', './data/train-tif/train_16074.tif')
('err', './data/train-tif/train_11386.tif')
('err', './data/train-tif/train_12871.tif')
('err', './data/train-tif/train_13674.tif')
('err', './data/train-tif/train_17739.tif')
('err', './data/train-tif/train_14454.tif')
('err', './data/train-tif/train_12138.tif')
('err', './data/train-tif/train_16902.tif')
('err', './data/train-tif/train_16075.tif')
('err', './data/train-tif/train_

('err', './data/train-tif/train_17759.tif')
('err', './data/train-tif/train_16920.tif')
('err', './data/train-tif/train_15262.tif')
('err', './data/train-tif/train_16094.tif')
('err', './data/train-tif/train_12157.tif')
('err', './data/train-tif/train_14472.tif')
('err', './data/train-tif/train_12885.tif')
('err', './data/train-tif/train_13689.tif')
('err', './data/train-tif/train_10679.tif')
('err', './data/train-tif/train_12886.tif')
('err', './data/train-tif/train_16095.tif')
('err', './data/train-tif/train_12158.tif')
('err', './data/train-tif/train_16921.tif')
('err', './data/train-tif/train_14473.tif')
('err', './data/train-tif/train_13690.tif')
('err', './data/train-tif/train_10680.tif')
('err', './data/train-tif/train_17760.tif')
('err', './data/train-tif/train_12887.tif')
('err', './data/train-tif/train_11405.tif')
('err', './data/train-tif/train_12159.tif')
('err', './data/train-tif/train_15263.tif')
('err', './data/train-tif/train_17761.tif')
('err', './data/train-tif/train_

('err', './data/train-tif/train_14489.tif')
('err', './data/train-tif/train_12903.tif')
('err', './data/train-tif/train_10699.tif')
('err', './data/train-tif/train_11421.tif')
('err', './data/train-tif/train_16114.tif')
('err', './data/train-tif/train_15284.tif')
('err', './data/train-tif/train_12179.tif')
('err', './data/train-tif/train_16937.tif')
('err', './data/train-tif/train_12904.tif')
('err', './data/train-tif/train_13705.tif')
('err', './data/train-tif/train_10700.tif')
('err', './data/train-tif/train_15285.tif')
('err', './data/train-tif/train_12180.tif')


In [22]:
for tags in tqdm(train.tags.values, miniters=1000):
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    y_train.append(targets)
    
y = np.array(y_train, np.uint8)

print('X.shape = ' + str(X.shape))
print('y.shape = ' + str(y.shape))

100%|██████████| 40479/40479 [00:00<00:00, 409333.33it/s]

X.shape = (40479, 21)
y.shape = (40479, 17)


In [46]:
n_classes = y.shape[1]

X_test = np.array(test_features.drop(['image_name', 'tags'], axis=1))

# Train and predict with one-vs-all strategy
y_pred = np.zeros((X_test.shape[0], n_classes))

print('Training and making predictions')

Xo = X[:,:18]
Xn = X[:,18:]
print Xo.shape
print Xn.shape
x = Xn[0]
print x[0].shape

Training and making predictions
(40479, 18)
(40479, 3)


AttributeError: 'list' object has no attribute 'shape'

In [24]:
for class_i in tqdm(range(n_classes), miniters=1): 
    model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                              silent=True, objective='binary:logistic', nthread=-1, \
                              gamma=0, min_child_weight=1, max_delta_step=0, \
                              subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                              base_score=0.5, seed=random_seed, missing=None)
    model.fit(X, y[:, class_i])
    y_pred[:, class_i] = model.predict_proba(X_test)[:, 1]

  0%|          | 0/17 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence.

In [16]:
preds = [' '.join(labels[y_pred_row > 0.2]) for y_pred_row in y_pred]

subm = pd.DataFrame()
subm['image_name'] = test_features.image_name.values
subm['tags'] = preds
subm.to_csv('submissionxgb.csv', index=False)